In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import trfp

import matplotlib
matplotlib.rc('xtick', labelsize=10)     
matplotlib.rc('ytick', labelsize=10)
matplotlib.rc('axes', titlesize=10)
matplotlib.rc('axes', labelsize=10)
plt.rcParams["font.family"] = "serif"

sns.set_style('darkgrid')


tr_x = np.array([0] + [1.75 * np.sin(2*np.pi/4*i) for i in np.arange(4)] + [3.5 * np.sin(2*np.pi/12*i) for i in np.arange(12)])
tr_y = np.array([0] + [-1.75 * np.cos(2*np.pi/4*i) for i in np.arange(4)] + [-3.5 * np.cos(2*np.pi/12*i) for i in np.arange(12)])

fp_x = np.array([-3, 0, 3, -3, 0, 3])
fp_x_offset = np.array([-4, -1, 2, -4, -1, 2])
fp_y = np.array([7.7, 7.7, 7.7, -7.7, -7.7, -7.7])

fp_x_4 = np.array([0, 3, 0, 3])
fp_y_4 = np.array([7.7, 7.7, -7.7, -7.7])

fp_x_4_st41 = np.array([3, 0, -3, 0])
fp_x_4_st37_st39 = np.array([0, 3, -3, 0])

def multipole(N,s,A,x,y):
    r = np.sqrt(x**2+y**2)
    theta = np.arctan2(y,x)
    if(s==0):
        By = 61.79e6 + A * (r/4.5)**N * np.cos(N*theta)
        Bx = A * (r/4.5)**N * np.sin(N*theta)
    if(s==1):
        By = 61.79e6 + A * (r/4.5)**N * np.sin(N*theta)
        Bx = -A * (r/4.5)**N * np.cos(N*theta)
    B = np.sqrt(Bx**2 + By**2) - 61.79e6
    return B

Ns = [0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8];
ss = [0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1];

THETA_TR = np.linalg.pinv(np.transpose(np.array([multipole(Ns[i],ss[i],1,tr_x,tr_y) for i in np.arange(17)])))

THETA_FP_6 = np.array([np.array([1, 1, 1, 1, 1, 1])/6.,  # dipole
                       np.array([1, 0, -1, 1, 0, -1])/-12.*4.5,  # n quad
                       np.array([1, 1, 1, -1, -1, -1])/46.2*4.5,  # s quad
                       np.array([1, 0, -1, -1, 0, 1])/-92.4/2.*4.5**2,  # s sext
                       np.array([1, -2, 1, 1, -2, 1])/18./2.*4.5**2,  # n sext
                       np.array([1, -2, 1, -1, 2, -1])/-138.6*4.5**3]  # NOT oct
                     )

THETA_FP_4 = np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2/2*4.5**2]  # sext
                     )

THETA_FP_4_ST37_ST39 = np.array([np.array([1, 0, 0, 1])/2.,  #dipole
                                 np.array([-1, 1, -1, 1])/6.*4.5,  # n quad
                                 np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                                 np.array([-1, 1, 1, -1])/46.2/2*4.5**2]  # sext?
                               )

THETA_FP_4_ST41 = np.array([np.array([0, 1, 0, 1])/2.,  #dipole
                            np.array([1, -1, -1, 1])/6.*4.5,  # n quad
                            np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                            np.array([1, -1, 1, -1])/46.2/2*4.5**2]  # sext?
                          )

In [2]:
multipole2 = trfp.matrices.__multipole

In [17]:
A = multipole(1,0,1,tr_x, tr_y)
B = multipole2(1,0,1,tr_x, tr_y)

np.matmul(trfp.THETA_TR, A) - np.matmul(THETA_TR, A)

array([ 2.63122337e-09, -2.66432043e-09, -1.31977762e-14, -3.87190280e-15,
       -7.88756740e-09,  2.03813215e-14,  3.13363369e-14, -2.98811337e-10,
       -3.92741395e-14,  3.07344536e-08,  7.30970839e-13,  5.63686714e-09,
        1.04621662e-18, -4.83865907e-08,  1.04360964e-12,  1.77635684e-15,
        9.71445147e-17])

array([ 1.87243072e-03, -2.05434389e-08, -2.70006240e-13,  1.42108547e-13,
       -4.04596133e-03,  3.16705382e-08, -1.61113242e-13, -1.43147479e-04,
        8.52651283e-14,  7.94957941e-07,  1.18234311e-11, -2.10036433e-11,
        0.00000000e+00, -1.31747629e-06,  2.09183781e-11,  0.00000000e+00,
        0.00000000e+00])

In [ ]:
Nss = [[0,0], [1,0], [1,1], [2,1], [2,0]]

J_6_PROBE = np.empty((5,5))
for i in range(5):
    Ns = Nss[i]

    B_fp = multipole(Ns[0], Ns[1], 1, fp_x, fp_y)
    J_6_PROBE[:, i] = np.matmul(THETA_FP_6, B_fp)[0:5]
J_6_PROBE = np.linalg.inv(J_6_PROBE)
J_6_PROBE[np.abs(J_6_PROBE)<1.0e-6] = 0

J_6_PROBE = np.round(J_6_PROBE, 3)

In [ ]:
matrix = trfp.matrices.J_6_PROBE_OFFSET

for row in matrix:
    row_str = [str(item) for item in np.round(row,3)]
    for i in range(len(row_str)):
        if row_str[i] == '0.0':
            row_str[i] = '0'
    print " & ".join(row_str) + r"\\"

In [ ]:
matrix = trfp.matrices.J_4_PROBE_ST41

for row in matrix:
    row_str = [str(item) for item in np.round(row,4)]
    for i in range(len(row_str)):
        if row_str[i] == '0.0':
            row_str[i] = '0'
    print " & ".join(row_str) + r"\\"